In [1]:
import pandas as pd
import numpy as np

In [2]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set


In [19]:
train['Pclass'].unique()

array([3, 1, 2], dtype=int64)

In [3]:
train["Age"] = train["Age"].fillna(train["Age"].median())


In [78]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,28.0,1,2,W./C. 6607,23.4500,NaN,0
889,890,1,1,"Behr, Mr. Karl Howell",0,26.0,0,0,111369,30.0000,C148,1


In [27]:
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train['Embarked'] == "C", "Embarked"] = 1
train.loc[train['Embarked'] == "Q", "Embarked"] = 2

test.loc[test["Embarked"] == "S", "Embarked"] = 0
test.loc[test['Embarked'] == "C", "Embarked"] = 1
test.loc[test['Embarked'] == "Q", "Embarked"] = 2

In [28]:
train.loc[train['Sex'] == 'male',"Sex"] = 0
train.loc[train['Sex'] == 'female','Sex'] = 1

test.loc[test['Sex'] == 'male',"Sex"] = 0
test.loc[test['Sex'] == 'female','Sex'] = 1

In [30]:
data

array([[ 3.,  0., 22.,  0.],
       [ 1.,  1., 38.,  1.],
       [ 3.,  1., 26.,  0.],
       ...,
       [ 3.,  1., 28.,  0.],
       [ 1.,  0., 26.,  1.],
       [ 3.,  0., 32.,  2.]])

In [31]:
data.shape

(891, 4)

In [81]:
data = np.array(train[["Pclass","Sex","Age","Embarked"]].values, dtype = float)
y = np.array(train['Survived'].values)
tdata = np.array(test[["Pclass","Sex","Age","Embarked"]].values, dtype = float)

# create model

In [32]:
data.shape

(891, 4)

In [33]:
data

array([[ 3.,  0., 22.,  0.],
       [ 1.,  1., 38.,  1.],
       [ 3.,  1., 26.,  0.],
       ...,
       [ 3.,  1., 28.,  0.],
       [ 1.,  0., 26.,  1.],
       [ 3.,  0., 32.,  2.]])

In [62]:
def preprocess_data(data,mi,mx):
    # data "Pclass","Sex","Age","Embarked"
    # Pclass = 1 2 3
    # sex = 0 1
    # embarked = 0 1 2
    age = (data[:,2]-mi)/mx
    isPclass1 = np.array([x== 1 for x in data[:,0]])
    isPclass2 = np.array([x== 2 for x in data[:,0]])
    isPclass3 = np.array([x== 3 for x in data[:,0]])
    isMale = np.array([x== 0 for x in data[:,1]])
    isFemale = np.array([x== 1 for x in data[:,1]])
    isEmbarked0 = np.array([x== 0 for x in data[:,3]])
    isEmbarked1 = np.array([x== 1 for x in data[:,3]])
    isEmbarked2 = np.array([x== 2 for x in data[:,3]])
    return np.vstack((age,isPclass1,isPclass2,isPclass3,isMale,isFemale,isEmbarked0,isEmbarked1,isEmbarked2)).T

In [86]:
mx = data[2].max()
mi = data[2].min()
new_data = preprocess_data(data,mi,mx)
test_data = preprocess_data(tdata,mi,mx)

In [87]:
new_data.shape

(891, 9)

In [88]:
vol_train = int(90*new_data.shape[0]/100)
val_data = new_data[vol_train:]
val_y = y[vol_train:]
train_data = new_data[:vol_train]
train_y = y[:vol_train]

In [90]:
train_data.shape

(801, 9)

In [96]:
def MSE(y,ypre):
    mse = sum([x**2 for x in (y-ypre)])/len(y)
    return mse

In [153]:
def acc(y,ypred,th):
    return (np.array([1 if ans>th else 0 for ans in ypred]) == y).sum()

In [175]:
def Bsigmoid(z):
    return 1.0/(1 + np.exp(-z))
sigmoid = np.vectorize(Bsigmoid)

In [173]:
def predict(weight,data):
    return sigmoid(weight[0] + np.sum(weight[1:]*data,axis = 1))

In [184]:
def precess(weight,x,y,lr,th):
    ypred = predict(weight,x)
    # update for MSE
    # w0 =  -2(ypred - y)
    # wx = -2(ypred-y)*Xx
    update_w0 = lr * -1 * 2 * (ypred - y).sum()
    update_wx = lr * -1 * 2 * np.sum((ypred - y) * x.T,axis=1)

    mse = MSE(y,ypred)
    accuracy = acc(y,ypred,th)
    return weight + np.append(update_w0,update_wx),mse,accuracy

In [185]:
epoch = 100
batch_size = 16
lr = 0.01
th=  0.5

In [190]:
weight = np.random.randint(-100,100,size = train_data.shape[1]+1)/1.0
weight

array([ 35.,  39.,  94., -62.,  48., -40., -93.,  27.,  62.,  12.])

In [191]:
for i in range(epoch):
    print(f"round {i+1}/epoch : ",end='')
    round = (train_data.shape[0]-1)//batch_size +1
    this_mse = 0
    this_acc = 0
    for j in range(round):
        batch_data = train_data[j*batch_size:(j+1)*batch_size]
        batch_y = train_y[j*batch_size:(j+1)*batch_size]
        weight,mse,accu = precess(weight,batch_data,batch_y,lr,th)
        this_mse+=mse
        this_acc+=accu
    print(f"Train MSE = {this_mse} : ",end = '')
    print(f"Train accuracy = : {this_acc/train_data.shape[0]} % : ",end = '')
    val_ypred = predict(weight,val_data)
    print(f"val MSE = {MSE(val_y,val_ypred)} : ",end = '')
    print(f"val accuracy = {acc(val_y,val_ypred,th)/val_data.shape[0]}")

round 1/epoch : Train MSE = 32.01879151008599 : Train accuracy = : 0.3595505617977528 % : val MSE = 0.5998360542376346 : val accuracy = 0.4
round 2/epoch : Train MSE = 30.52453912155684 : Train accuracy = : 0.3895131086142322 % : val MSE = 0.5646816898145353 : val accuracy = 0.4222222222222222
round 3/epoch : Train MSE = 30.238159890201683 : Train accuracy = : 0.3895131086142322 % : val MSE = 0.5299849874411381 : val accuracy = 0.4666666666666667
round 4/epoch : Train MSE = 27.015373373017617 : Train accuracy = : 0.4469413233458177 % : val MSE = 0.3354683530943087 : val accuracy = 0.6444444444444445
round 5/epoch : Train MSE = 19.140153562165324 : Train accuracy = : 0.6067415730337079 % : val MSE = 0.31087142554973846 : val accuracy = 0.6666666666666666
round 6/epoch : Train MSE = 18.391677754811923 : Train accuracy = : 0.6217228464419475 % : val MSE = 0.29619715420286524 : val accuracy = 0.6888888888888889
round 7/epoch : Train MSE = 18.156669424442264 : Train accuracy = : 0.632958801

In [199]:
submission_y = np.array([1 if ans>th else 0 for ans in predict(weight,test_data)])

In [200]:
submission_y.shape

(418,)

In [210]:
submission = pd.DataFrame({"PassengerId":test['PassengerId'].values,'Survived':submission_y})

In [212]:
submission.to_csv('submission.csv',index=False)